In [6]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler

In [7]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [8]:
#train.isnull().sum()

In [9]:
#test.isnull().sum()

In [10]:
# print('train 계약년월 기간')
# print(train['계약년월'].min())
# print(train['계약년월'].max())
# print('test 계약년월 기간')
# print(test['계약년월'].min())
# print(test['계약년월'].max())

### 변수 가공

In [11]:
# 시계열 반영 > 연/월/분기로 분류
train['계약년월'] = pd.to_datetime(train['계약년월'], format='%Y%m')
train['계약년'] = train['계약년월'].dt.year
train['계약월'] = train['계약년월'].dt.month
train['계약분기'] = train['계약년월'].dt.quarter # 분기 (1~4)

# 월을 기준으로 계절 column 생성
def get_season(month):
    if month in [3,4,5]:
        return '봄'
    elif month in [6,7,8]:
        return '여름'
    elif month in [9,10,11]:
        return '가을'
    else:
        return '겨울'
train['계절'] = train['계약월'].apply(get_season)

# one-hot encoding을 위해 season 숫자형 매핑
season_map = {'봄':0, '여름':1, '가을':2, '겨울':3}
train['계절코드'] = train['계절'].map(season_map)

In [12]:
# 시계열 반영 > 연/월/분기로 분류
test['계약년월'] = pd.to_datetime(test['계약년월'], format='%Y%m')
test['계약년'] = test['계약년월'].dt.year
test['계약월'] = test['계약년월'].dt.month
test['계약분기'] = test['계약년월'].dt.quarter  # 분기 (1~4)

# 월을 기준으로 계절 column 생성
def get_season(month):
    if month in [3,4,5]:
        return '봄'
    elif month in [6,7,8]:
        return '여름'
    elif month in [9,10,11]:
        return '가을'
    else:
        return '겨울'
    
test['계절'] = test['계약월'].apply(get_season)

# one-hot encoding을 위해 season 숫자형 매핑
season_map = {'봄':0, '여름':1, '가을':2, '겨울':3}
test['계절코드'] = test['계절'].map(season_map)

In [13]:
# # '구' 데이터 값 확인
# print(train['구'].unique())
# print(test['구'].unique())

In [14]:
CBD = {'종로구','중구','용산구'}
GANGNAM3 = {'강남구','서초구','송파구'}
INNER = {'마포구','성동구','광진구','동대문구','서대문구','성북구','영등포구','양천구','동작구','관악구','강동구'}
OUTER = {'강서구','구로구','금천구','강북구','도봉구','노원구','은평구','중랑구'}

def map_zone(gu):
    if gu in CBD: return '도심'
    if gu in GANGNAM3: return '강남3'
    if gu in INNER: return '내부권'
    return '외곽'

# train, test 둘 다 동일하게 적용
for df in [train, test]:
    df['zone4'] = df['구'].map(map_zone)
    zone_dummies = pd.get_dummies(df['zone4'], prefix='zone4')
    df[zone_dummies.columns] = zone_dummies

In [15]:
# 연식 별로 신축 or 재건축 라벨링
for df in [train, test]:
    df['신축(10년 미만)'] = (df['연식'] < 10).astype(int)      # 신축
    df['재건축 연한(30년 이상)'] = (df['연식'] >= 30).astype(int) # 재건축 연한

In [16]:
school = pd.read_csv('school.csv')
#school[school['학교종류명'] == '고등학교']

In [17]:
# display(train[['x좌표', 'y좌표']])
# display(school[['위도', '경도']])

In [18]:
# 학군지 데이터 파생변수
EARTH_R = 6371000.0

# 좌표 > np 값으로 변환
def _to_rad(lat_arr, lon_arr):
    return np.c_[np.deg2rad(lat_arr), np.deg2rad(lon_arr)]

# 거리 계산
def _count_in_radius(tree, pts_rad, radius_m, batch=200_000):
    r = radius_m / EARTH_R
    out = np.empty(pts_rad.shape[0], dtype=np.int32)
    for i in range(0, pts_rad.shape[0], batch):
        ind = tree.query_radius(pts_rad[i:i+batch], r=r)
        out[i:i+batch] = [len(ix) for ix in ind]
    return out

In [19]:
# 학교까지 최근접 거리 구하기

# school 원본에서 급별 분리 (열 이름: '학교종류명', '위도', '경도')
elem_df = school[school['학교종류명'] == '초등학교'][['위도','경도']].dropna().copy()
mid_df  = school[school['학교종류명'] == '중학교'][['위도','경도']].dropna().copy()
high_df = school[school['학교종류명'] == '고등학교'][['위도','경도']].dropna().copy()

# 각 급별 BallTree (하버사인 거리)
tree_elem = BallTree(_to_rad(elem_df['위도'].values, elem_df['경도'].values), metric='haversine')
tree_mid  = BallTree(_to_rad(mid_df['위도'].values,  mid_df['경도'].values),  metric='haversine')
tree_high = BallTree(_to_rad(high_df['위도'].values, high_df['경도'].values), metric='haversine')

def add_school_features(df, tree, prefix, radii=(500, 1000, 1500, 2000),
                        lat_col='y좌표', lon_col='x좌표'):
    pts = _to_rad(df[lat_col].values, df[lon_col].values)

    # 1) 최근접 거리 (m)
    dist_rad, _ = tree.query(pts, k=1)
    df[f'{prefix}_min_dist_m'] = dist_rad[:, 0] * EARTH_R
    df[f'{prefix}_min_dist_log'] = np.log1p(df[f'{prefix}_min_dist_m'])

    # 2) 반경 내 개수
    for r in radii:
        df[f'{prefix}_cnt_{r}m'] = _count_in_radius(tree, pts, r)

    return df

# train / test 모두에 적용
for d in (train, test):
    add_school_features(d, tree_elem, prefix='elem')  # 초등
    add_school_features(d, tree_mid,  prefix='mid')   # 중등
    add_school_features(d, tree_high, prefix='high')  # 고등

In [20]:
# 명문/특수 카테고리 집합 (데이터셋 라벨에 맞춰 유연하게 확장)
elite_cats = {
    '특목고', '자율형사립고', '자사고', '과학고', '영재학교', '국제고', '외국어고', '마이스터고'
}

elite_df = (school
    .query("학교종류명 == '고등학교'")
    .loc[lambda x: x['고등학교 구분명'].isin(elite_cats), ['위도','경도']]
    .dropna()
    .copy()
)

# 엘리트 고교가 하나도 없는 경우를 대비한 방어
if len(elite_df):
    tree_elite = BallTree(_to_rad(elite_df['위도'].values, elite_df['경도'].values), metric='haversine')

    def add_proximity_dummy(df, tree, radius_m=2000, lat_col='y좌표', lon_col='x좌표', name='elite2k'):
        pts = _to_rad(df[lat_col].values, df[lon_col].values)
        counts = _count_in_radius(tree, pts, radius_m)
        df[f'is_{name}'] = (counts > 0).astype(int)
        return df

    for d in (train, test):
        add_proximity_dummy(d, tree_elite, radius_m=2000, name='elite2k')
        # 원하면 반경을 여러 개(1500m/2000m)로 만들어 비교도 가능:
        # add_proximity_dummy(d, tree_elite, radius_m=1500, name='elite1_5k')
else:
    # 엘리트 표가 비어있으면 0으로 채움
    for d in (train, test):
        d['is_elite2k'] = 0

In [21]:
def to_rad(lat, lon):
    return np.c_[np.deg2rad(lat), np.deg2rad(lon)]

def make_coord_key(df, lat_col='y좌표', lon_col='x좌표', ndigits=6):
    return (df[lat_col].round(ndigits).astype(str) + '_' +
            df[lon_col].round(ndigits).astype(str))

# 0) 좌표 키 만들기 (train+test 통합 → 유니크 좌표에서만 계산)
for df in (train, test):
    df['coord_key'] = make_coord_key(df)

coords = (pd.concat([train[['coord_key','y좌표','x좌표']],
                     test[['coord_key','y좌표','x좌표']]], axis=0)
            .drop_duplicates('coord_key')
            .reset_index(drop=True))

pts_rad = to_rad(coords['y좌표'].values, coords['x좌표'].values)

# 1) 엘리트 고교 서브셋: 특목고 ∪ (자율고 & 사립)
high = (school.query("학교종류명 == '고등학교'")
              [['설립구분','고등학교 구분명','위도','경도']]
              .dropna(subset=['위도','경도'])
              .copy())

elite_mask = (high['고등학교 구분명'].eq('특목고')) | \
             (high['고등학교 구분명'].eq('자율고') & high['설립구분'].eq('사립'))
elite_df = high.loc[elite_mask, ['위도','경도']].copy()

# 2) 엘리트 트리로 거리/카운트 계산
feats = coords[['coord_key']].copy()

if len(elite_df):
    elite_rad = to_rad(elite_df['위도'].values, elite_df['경도'].values)
    tree = BallTree(elite_rad, metric='haversine', leaf_size=60)

    # 최근접 거리 (m)
    dist_rad, _ = tree.query(pts_rad, k=1)
    dist_m = dist_rad[:, 0] * EARTH_R
    feats['elite_min_dist_m']   = dist_m
    feats['elite_min_dist_log'] = np.log1p(dist_m)

    # 반경 내 개수 (1.5km, 2km)
    cnt_15 = tree.query_radius(pts_rad, r=1500/ EARTH_R, count_only=True).astype(np.int32)
    cnt_20 = tree.query_radius(pts_rad, r=2000/ EARTH_R, count_only=True).astype(np.int32)
    feats['elite_cnt_1.5k'] = cnt_15
    feats['elite_cnt_2k']   = cnt_20

    # 2km 이내 여부
    feats['is_elite_2k'] = (cnt_20 > 0).astype(np.int8)

# 3) train/test에 매핑
train = train.merge(feats, on='coord_key', how='left')
test  = test.merge(feats,  on='coord_key',  how='left')

In [22]:
# print(train['아파트명'].unique())
# print(test['아파트명'].unique())

In [23]:
# 아파트 브랜드 > binary(0/1)로 표시
for df in (train, test):
    dmy = pd.get_dummies(df['아파트명'], prefix='brand')
    df[dmy.columns] = dmy

# 열 정렬(혹시 누락 생기면 0으로 보정)
brand_cols = [c for c in train.columns if c.startswith('brand_')]
for c in brand_cols:
    if c not in test.columns: test[c] = 0
for c in [c for c in test.columns if c.startswith('brand_')]:
    if c not in train.columns: train[c] = 0

In [24]:
# 아파트 브랜드 개수 구하기
vc = train['아파트명'].value_counts()
train['apt_brand_count'] = train['아파트명'].map(vc).astype(int)
test['apt_brand_count']  = test['아파트명'].map(vc).fillna(0).astype(int)

In [25]:
# 계약년월, 아파트 브랜드 별 평균가(target)
def target_encode_timeseries(
    train, test, cat_col='아파트명', target_col='target', time_col='계약년월',
    n_splits=5, alpha=20, noise=0.01, seed=42
):
    tr = train.sort_values(time_col).copy()
    te = test.copy()
    rng = np.random.RandomState(seed)

    global_mean = tr[target_col].mean()
    te_col = f'{cat_col}_TE'
    tr[te_col] = np.nan

    # 월 단위 고유 순서로 TSS
    months = tr[time_col].drop_duplicates().sort_values().to_list()
    tss = TimeSeriesSplit(n_splits=n_splits)
    idx = np.arange(len(months))

    for tr_idx, val_idx in tss.split(idx):
        tr_months = set(months[i] for i in tr_idx)
        val_mask = tr[time_col].isin(set(months[i] for i in val_idx))
        base = tr[tr[time_col].isin(tr_months)]

        stats = base.groupby(cat_col)[target_col].agg(['mean','count'])
        # m-Estimate smoothing
        stats['te'] = (stats['mean']*stats['count'] + global_mean*alpha) / (stats['count'] + alpha)

        mapping = stats['te']
        tr.loc[val_mask, te_col] = tr.loc[val_mask, cat_col].map(mapping).fillna(global_mean)

    # 노이즈로 과적합 방지
    tr[te_col] = tr[te_col] * (1 + noise * rng.randn(len(tr)))

    # 최종 매핑(전체 train으로 fit → test에 적용)
    stats_full = tr.groupby(cat_col)[target_col].agg(['mean','count'])
    stats_full['te'] = (stats_full['mean']*stats_full['count'] + global_mean*alpha) / (stats_full['count'] + alpha)
    mapping_full = stats_full['te']

    # 결측치 global_mean 값으로 fill
    tr[te_col] = tr[te_col].fillna(tr[cat_col].map(mapping_full)).fillna(global_mean)

    te[te_col] = te[cat_col].map(mapping_full).fillna(global_mean)
    return tr, te

# 사용 예
train, test = target_encode_timeseries(
    train, test, cat_col='아파트명', target_col='target', time_col='계약년월')

### 스케일 정리 (log)

In [36]:
def summarize_numeric(
    df: pd.DataFrame,
    exclude_cols=('target',),           # 필요시 추가
    skip_binary=True,                   # 0/1 더미 제외
    skip_patterns=(r'^brand_', r'^zone4_', r'^is_', r'_더미$', r'_dummy$'),
    quantiles=(0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99)
) -> pd.DataFrame:
    cols = []
    for c in df.columns:
        if c in exclude_cols: 
            continue
        if not is_numeric_dtype(df[c]): 
            continue
        if any(re.search(pat, c) for pat in skip_patterns):
            continue
        s = df[c]
        # 스칼라만(전부 NaN은 스킵)
        if s.notna().sum() == 0: 
            continue
        # 0/1 더미 제외
        if skip_binary and s.dropna().nunique() <= 2:
            continue
        # 이미 로그/지수변환 컬럼은 제외
        if any(k in c for k in ['log', 'LOG', 'ln_', '_ln']):
            continue
        cols.append(c)

    rows = []
    for c in cols:
        s = pd.to_numeric(df[c], errors='coerce')
        q = s.quantile(list(quantiles))
        n = s.size
        na = s.isna().sum()
        nz = (s==0).sum()
        uniq = s.nunique(dropna=True)
        vmin, vmax = s.min(), s.max()
        mean, std = s.mean(), s.std()
        skew = s.skew()  # pandas skew (Fisher)
        # 간단한 로그 추천 규칙
        nonneg = (vmin >= 0)
        spread = (q.loc[0.95] / (q.loc[0.05] + 1e-9)) if pd.notna(q.loc[0.05]) and q.loc[0.05] != 0 else np.inf
        log_reco = (nonneg and (skew is not None) and (skew > 1.0) and (spread > 20))
        sqrt_reco = (nonneg and not log_reco and (skew is not None) and (0.7 < skew <= 1.0))
        yeojohnson_reco = (vmin < 0 and (skew is not None) and (skew > 1.0))
        winsorize_reco = (np.isfinite(spread) and spread > 50)

        rows.append({
            'col': c,
            'count': n,
            'na_cnt': na,
            'na_rate': na / n,
            'unique': uniq,
            'min': vmin, 'p01': q.iloc[0], 'p05': q.iloc[1], 'p25': q.iloc[2],
            'p50': q.iloc[3], 'p75': q.iloc[4], 'p95': q.iloc[5], 'p99': q.iloc[6],
            'max': vmax,
            'mean': mean, 'std': std, 'skew': skew,
            'share_zero': nz / n,
            'non_negative': nonneg,
            'spread(p95/p05)': spread,
            'recommend': ('log1p' if log_reco else
                          'sqrt' if sqrt_reco else
                          'yeo-johnson' if yeojohnson_reco else
                          'none'),
            'also_clip?': winsorize_reco
        })
    out = pd.DataFrame(rows).sort_values(['recommend','skew','spread(p95/p05)'], ascending=[True, False, False])
    # 로그 후보만 한 번 더 보기 좋게 맨 위로
    out = out.sort_values(by=['recommend','skew'], ascending=[True, False]).reset_index(drop=True)
    return out

summary = summarize_numeric(train)

# 로그 변환 추천 리스트 뽑기
log_candidates = summary.query("recommend == 'log1p'")['col'].tolist()
sqrt_candidates = summary.query("recommend == 'sqrt'")['col'].tolist()
yeo_candidates = summary.query("recommend == 'yeo-johnson'")['col'].tolist()

print("🔹 log1p 후보:", log_candidates[:20])
print("🔸 sqrt 후보 :", sqrt_candidates[:20])
print("🟡 yeo-johnson 후보:", yeo_candidates[:20])

# 상위 치우침(왜도) 컬럼 TOP 20 미리 보기
display(summary[['col','na_rate','min','p05','p50','p95','max','skew','share_zero','recommend']].head(20))

🔹 log1p 후보: ['high_cnt_500m', 'elite_cnt_1.5k']
🔸 sqrt 후보 : ['버스정류장수', 'elite_cnt_2k', '지하철수', 'high_cnt_1000m']
🟡 yeo-johnson 후보: []


,col,na_rate,min,p05,p50,p95,max,skew,share_zero,recommend
0,high_cnt_500m,0.0,0.000000,0.000000,0.000000,2.000000,7.000000,1.365219,0.547997,log1p
1,elite_cnt_1.5k,0.0,0.000000,0.000000,0.000000,2.000000,4.000000,1.250261,0.607413,log1p
2,버스거리,0.0,0.000830,0.042059,0.115043,0.244755,33.026164,50.408599,0.000000,none
3,elem_min_dist_m,0.0,2.985071,124.350291,292.811374,616.070162,36549.634421,43.246596,0.000000,none
4,초등학교거리,0.0,0.002985,0.124350,0.292811,0.616070,36.549634,43.246596,0.000000,none
5,mid_min_dist_m,0.0,38.325663,153.814597,416.453554,893.220999,36362.290555,34.771109,0.000000,none
6,high_min_dist_m,0.0,34.625313,179.679580,537.450161,1262.709402,36892.514957,25.894747,0.000000,none
7,지하철거리,0.0,0.018769,0.168667,0.479064,1.232623,16.953702,4.170422,0.000000,none
8,elite_min_dist_m,0.0,77.934272,488.249191,1808.304321,3669.624577,37765.634960,3.997011,0.000000,none
9,회사채금리t6,0.0,1.645000,1.781000,2.179000,2.926000,5.487000,3.455257,0.000000,none


In [35]:
def _safe_log1p_col(df, col, new_name=None):
    if col not in df: 
        return
    if new_name is None:
        # 점(.) 들어간 컬럼명은 새 이름에서 'p'로 치환
        new_name = f"{col.replace('.', 'p')}_log"
    x = pd.to_numeric(df[col], errors='coerce')
    df[new_name] = np.log1p(np.clip(x, a_min=0, a_max=None))

def add_transformations(train, test):
    # 1) log1p로 반드시 변환(추천)
    log_counts = [
        'high_cnt_500m', 'elite_cnt_1.5k',  # 요약표에서 log1p 후보
        'elite_cnt_2k', 'high_cnt_1000m',  # sqrt 후보지만 일관성 위해 log 추천
        '버스정류장수', '지하철수'           # sqrt 후보지만 카운트류로 log 추천
    ]

    for df in (train, test):
        for c in log_counts:
            _safe_log1p_col(df, c)

    # 2) 거리 → log1p (거리 감쇠를 모델이 쉽게 학습)
    for df in (train, test):
        _safe_log1p_col(df, '버스거리')
        _safe_log1p_col(df, '지하철거리')

    # 3) 감쇠 접근성 지수(강추)
    for df in (train, test):
        if '지하철거리' in df and 'subway_access_idx' not in df:
            df['subway_access_idx'] = np.exp(-pd.to_numeric(df['지하철거리'], errors='coerce') / 700.0)
        if '버스거리' in df and 'bus_access_idx' not in df:
            df['bus_access_idx'] = np.exp(-pd.to_numeric(df['버스거리'], errors='coerce') / 250.0)

        # 학교 접근성 지수(거리 로그 이미 있어도 감쇠지수 추가는 유효)
        for prefix, beta in [('elem',500.0), ('mid',600.0), ('high',700.0)]:
            dcol = f'{prefix}_min_dist_m'
            acol = f'{prefix}_access'
            if dcol in df and acol not in df:
                df[acol] = np.exp(-pd.to_numeric(df[dcol], errors='coerce') / beta)

        # 통합 교육 접근성(원하면 가중합은 데이터로 학습해도 OK; 일단 기본 가중치)
        if {'elem_access','mid_access','high_access'}.issubset(df.columns) and 'edu_access_idx' not in df:
            df['edu_access_idx'] = (
                1.0*df['elem_access'] + 0.6*df['mid_access'] + 0.5*df['high_access']
            )

    return train, test

train, test = add_transformations(train, test)

### 최종 데이터 정보

In [33]:
train.info()
train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 635467 entries, 567662 to 106198
Columns: 104 entries, 전용면적(㎡) to edu_access_idx
dtypes: datetime64[ns](1), float64(47), int32(14), int64(15), int8(1), object(9), uint8(17)
memory usage: 398.8+ MB


전용면적(㎡)           0
계약년월              0
층                 0
건축년도              0
구                 0
                 ..
bus_access_idx    0
elem_access       0
mid_access        0
high_access       0
edu_access_idx    0
Length: 104, dtype: int64

In [29]:
test.info()
test.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9272 entries, 0 to 9271
Columns: 103 entries, 전용면적(㎡) to edu_access_idx
dtypes: datetime64[ns](1), float64(47), int32(14), int64(14), int8(1), object(9), uint8(17)
memory usage: 5.7+ MB


전용면적(㎡)           0
계약년월              0
층                 0
건축년도              0
구                 0
                 ..
bus_access_idx    0
elem_access       0
mid_access        0
high_access       0
edu_access_idx    0
Length: 103, dtype: int64

In [30]:
print('<train 변수>')
for i, c in enumerate(train.columns, 1):
    print(f'{i:3d}. {c}')

<train 변수>
  1. 전용면적(㎡)
  2. 계약년월
  3. 층
  4. 건축년도
  5. 구
  6. 회사채금리
  7. 매매가격지수
  8. 건설공사비지수
  9. 거래량
 10. 버스정류장수
 11. 지하철수
 12. 연식
 13. 강남권여부
 14. 아파트명
 15. x좌표
 16. y좌표
 17. 대장아파트거리
 18. 버스거리
 19. 지하철거리
 20. 지하철접근성
 21. 초등학교거리
 22. 초등학교거리구분
 23. 1km이내학교수
 24. 고등학교진학률
 25. 회사채금리t3
 26. 회사채금리t6
 27. 회사채금리t12
 28. delta3
 29. delta6
 30. delta12
 31. MA3
 32. MA6
 33. 전용면적(log)
 34. 전용면적구간
 35. 평수
 36. 대장아파트거리(log)
 37. 대장아파트거리접근성
 38. target
 39. 계약년
 40. 계약월
 41. 계약분기
 42. 계절
 43. 계절코드
 44. zone4
 45. zone4_강남3
 46. zone4_내부권
 47. zone4_도심
 48. zone4_외곽
 49. 신축(10년 미만)
 50. 재건축 연한(30년 이상)
 51. elem_min_dist_m
 52. elem_min_dist_log
 53. elem_cnt_500m
 54. elem_cnt_1000m
 55. elem_cnt_1500m
 56. elem_cnt_2000m
 57. mid_min_dist_m
 58. mid_min_dist_log
 59. mid_cnt_500m
 60. mid_cnt_1000m
 61. mid_cnt_1500m
 62. mid_cnt_2000m
 63. high_min_dist_m
 64. high_min_dist_log
 65. high_cnt_500m
 66. high_cnt_1000m
 67. high_cnt_1500m
 68. high_cnt_2000m
 69. is_elite2k
 70. coord_key
 71. eli

In [31]:
print('<test 변수>')
for i, c in enumerate(test.columns, 1):
    print(f'{i:3d}. {c}')

<test 변수>
  1. 전용면적(㎡)
  2. 계약년월
  3. 층
  4. 건축년도
  5. 구
  6. 회사채금리
  7. 매매가격지수
  8. 건설공사비지수
  9. 거래량
 10. 버스정류장수
 11. 지하철수
 12. 연식
 13. 강남권여부
 14. 아파트명
 15. x좌표
 16. y좌표
 17. 대장아파트거리
 18. 버스거리
 19. 지하철거리
 20. 지하철접근성
 21. 초등학교거리
 22. 초등학교거리구분
 23. 1km이내학교수
 24. 고등학교진학률
 25. 회사채금리t3
 26. 회사채금리t6
 27. 회사채금리t12
 28. delta3
 29. delta6
 30. delta12
 31. MA3
 32. MA6
 33. 전용면적(log)
 34. 전용면적구간
 35. 평수
 36. 대장아파트거리(log)
 37. 대장아파트거리접근성
 38. 계약년
 39. 계약월
 40. 계약분기
 41. 계절
 42. 계절코드
 43. zone4
 44. zone4_강남3
 45. zone4_내부권
 46. zone4_도심
 47. zone4_외곽
 48. 신축(10년 미만)
 49. 재건축 연한(30년 이상)
 50. elem_min_dist_m
 51. elem_min_dist_log
 52. elem_cnt_500m
 53. elem_cnt_1000m
 54. elem_cnt_1500m
 55. elem_cnt_2000m
 56. mid_min_dist_m
 57. mid_min_dist_log
 58. mid_cnt_500m
 59. mid_cnt_1000m
 60. mid_cnt_1500m
 61. mid_cnt_2000m
 62. high_min_dist_m
 63. high_min_dist_log
 64. high_cnt_500m
 65. high_cnt_1000m
 66. high_cnt_1500m
 67. high_cnt_2000m
 68. is_elite2k
 69. coord_key
 70. elite_min_dist_m

In [34]:
# 전체 train / test 저장
train.to_csv('train_all_feature.csv', index=False, encoding='utf-8-sig')
test.to_csv('test_all_feature.csv', index=False, encoding='utf-8-sig')